# Before You Begin: Depression Detection Analysis
1. Accessing the Data
All preprocessed ZIP files, including audio, text transcripts, and OpenFace CSVs, can be found here:
[Google Drive Folder](https://drive.google.com/drive/folders/1a1fsP2zifaadfY3FNF9zWLt5ekcLigsn?usp=sharing)

Add a shortcut of this folder to your My Drive by clicking “Add shortcut to Drive.”

2. Setting Up the Notebook
Open the Colab notebook and mount your Google Drive.

Skip the sections on transcript generation and heavy preprocessing — these steps have already been completed.

In the Data Loading section, make sure to update the file path to match the location of the Drive shortcut.

Run the unzip cell to extract the dataset (Section - Extracting Feature Folders (Audio and Facial)).

3. Loading Features & Training the Model
Execute the cells that load the following pre-computed features:

MFCC

Transcript embeddings

Facial Action Units (OpenFace CSVs)

Then, run the cells that define and train the CNN–BiLSTM model.

Note: The training uses data from 105 participants due to GPU limitations.

4. Evaluating the Model & Running Inference
Once training is complete, run the evaluation section to:

View the detection accuracy

Calculate the Mean Absolute Error (MAE) for detection prediction

# Imports and Installations

In [2]:
# Mount Google Drive to access and store data
from google.colab import drive
drive.mount('/content/drive')

# Install necessary packages
!pip install openai-whisper
!pip install pydub
!pip install google-generativeai
!pip install tensorflow
!pip install sentence-transformers

# System and File Handling
import os
import csv
from datetime import timedelta

# Audio Processing
# import whisper
# from pydub import AudioSegment

# Google Generative AI (if used)
import google.generativeai as genai

# Numerical and Data Handling
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# Deep Learning: TensorFlow and Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Input, Conv1D, BatchNormalization, Dropout,
                                     Bidirectional, LSTM, GlobalMaxPooling1D, Dense)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Sentence Embeddings
from sentence_transformers import SentenceTransformer

# Visualization
import matplotlib.pyplot as plt

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Segmented Transcript generation from raw Audio files

In [2]:
# Load the Whisper ASR model
asr_model = whisper.load_model("small")

def generate_transcription(audio_file):
    """
    Transcribes the given audio using Whisper and returns segment details.
    """
    result = asr_model.transcribe(audio_file)
    return result["segments"]

def correct_transcription(text):
    """
    Cleans up ASR output by fixing errors and removing repetitions.
    """
    prompt = (
        "This transcription may have repetition and inaccuracies. "
        "Please correct all errors and eliminate duplicate parts:\n\n"
        f"\"{text}\"\n\n"
        "Only return the corrected sentence."
    )
    response = model.generate_content(prompt)
    return response.text.strip()

def transcribe_multiple_files(source_dir, output_dir="/content/drive/MyDrive/CGS_Audio_FacialData/transcripts", start_index=367, end_index=440):
    """
    Iteratively processes audio files within a numeric range and saves transcripts.
    """
    os.makedirs(output_dir, exist_ok=True)

    for idx in range(start_index, end_index):
        audio_filename = f"{idx}_AUDIO.wav"
        audio_path = os.path.join(source_dir, audio_filename)

        if os.path.isfile(audio_path):
            output_csv = os.path.join(output_dir, f"{idx}_transcript.csv")
            try:
                print(f"Transcribing {audio_filename}...")
                audio_to_csv(audio_path, csv_path=output_csv)
            except Exception as err:
                print(f"Failed to process {audio_filename}: {err}")
        else:
            print(f"Missing audio file: {audio_filename}")

# Run the batch processing pipeline
transcribe_multiple_files("/content/")

# Backup transcripts to Drive (if needed)
!cp -r /content/transcripts /content/drive/MyDrive/CGS_Audio_FacialData/

100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 59.8MiB/s]


⚠️ Missing audio file: 367_AUDIO.wav
⚠️ Missing audio file: 368_AUDIO.wav
⚠️ Missing audio file: 369_AUDIO.wav
⚠️ Missing audio file: 370_AUDIO.wav
⚠️ Missing audio file: 371_AUDIO.wav
⚠️ Missing audio file: 372_AUDIO.wav
⚠️ Missing audio file: 373_AUDIO.wav
⚠️ Missing audio file: 374_AUDIO.wav
⚠️ Missing audio file: 375_AUDIO.wav
⚠️ Missing audio file: 376_AUDIO.wav
⚠️ Missing audio file: 377_AUDIO.wav
⚠️ Missing audio file: 378_AUDIO.wav
⚠️ Missing audio file: 379_AUDIO.wav
⚠️ Missing audio file: 380_AUDIO.wav
⚠️ Missing audio file: 381_AUDIO.wav
⚠️ Missing audio file: 382_AUDIO.wav
⚠️ Missing audio file: 383_AUDIO.wav
⚠️ Missing audio file: 384_AUDIO.wav
⚠️ Missing audio file: 385_AUDIO.wav
⚠️ Missing audio file: 386_AUDIO.wav
⚠️ Missing audio file: 387_AUDIO.wav
⚠️ Missing audio file: 388_AUDIO.wav
⚠️ Missing audio file: 389_AUDIO.wav
⚠️ Missing audio file: 390_AUDIO.wav
⚠️ Missing audio file: 391_AUDIO.wav
⚠️ Missing audio file: 392_AUDIO.wav
⚠️ Missing audio file: 393_AUDIO.wav
⚠

In [3]:
def save_audio_clip(audio_file, start_time, end_time, destination):
    """
    Extracts a portion of the WAV file and saves it as a new audio segment.
    """
    audio = AudioSegment.from_wav(audio_file)
    clip = audio[start_time * 1000 : end_time * 1000]
    clip.export(destination, format="wav")

def audio_to_csv(audio_file, csv_path="segments.csv"):
    """
    Converts an audio file into a CSV containing transcript with timestamps.
    """
    segments = generate_transcription(audio_file)

    with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['text', 'start', 'end'])
        writer.writeheader()

        for segment in segments:
            writer.writerow({
                'text': segment['text'],
                'start': segment['start'],
                'end': segment['end']
            })

    print(f"Transcript saved: {csv_path}")

In [4]:
model_whisper = whisper.load_model("small")

def transcribe_audio(audio_path):
    result = model_whisper.transcribe(audio_path)
    return result["segments"]  # Each has 'text', 'start', 'end'

def clean_transcription(segment_text):
    prompt = (
        "The following transcription has possible mistakes and repeated phrases. "
        "Please correct transcription errors and remove duplicate content:\n\n"
        f"\"{segment_text}\"\n\n"
        "Return only the cleaned text."
    )
    response = model.generate_content(prompt)
    return response.text.strip()

def extract_segment(audio_path, start_sec, end_sec, output_path):
    audio = AudioSegment.from_wav(audio_path)
    segment = audio[start_sec * 1000 : end_sec * 1000]  # milliseconds
    segment.export(output_path, format="wav")

In [5]:
def process_audio_pipeline(audio_path, output_csv_path="segments.csv"):
    segments = transcribe_audio(audio_path)

    # Open CSV file for writing
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['text', 'start', 'end'])
        writer.writeheader()

        for i, seg in enumerate(segments):
            writer.writerow({
                'text': seg['text'],
                'start': seg['start'],
                'end': seg['end']
            })

    print(f"\n📄 CSV saved at: {output_csv_path}")


In [6]:
def batch_process_audio(folder_path, output_folder="/content/drive/MyDrive/CGS_Audio_FacialData/transcripts"):
    os.makedirs(output_folder, exist_ok=True)

    for i in range(367, 440):  # 719 because range is exclusive on the upper end
        file_name = f"{i}_AUDIO.wav"
        file_path = os.path.join(folder_path, file_name)

        if os.path.exists(file_path):
            output_csv = os.path.join(output_folder, f"{i}_transcript.csv")
            try:
                print(f"Processing {file_name}...")
                process_audio_pipeline(file_path, output_csv_path=output_csv)
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
        else:
            print(f"File not found: {file_name}")

# 🔧 Usage
batch_process_audio("/content/")


⚠️ File not found: 367_AUDIO.wav
⚠️ File not found: 368_AUDIO.wav
⚠️ File not found: 369_AUDIO.wav
⚠️ File not found: 370_AUDIO.wav
⚠️ File not found: 371_AUDIO.wav
⚠️ File not found: 372_AUDIO.wav
⚠️ File not found: 373_AUDIO.wav
⚠️ File not found: 374_AUDIO.wav
⚠️ File not found: 375_AUDIO.wav
⚠️ File not found: 376_AUDIO.wav
⚠️ File not found: 377_AUDIO.wav
⚠️ File not found: 378_AUDIO.wav
⚠️ File not found: 379_AUDIO.wav
⚠️ File not found: 380_AUDIO.wav
⚠️ File not found: 381_AUDIO.wav
⚠️ File not found: 382_AUDIO.wav
⚠️ File not found: 383_AUDIO.wav
⚠️ File not found: 384_AUDIO.wav
⚠️ File not found: 385_AUDIO.wav
⚠️ File not found: 386_AUDIO.wav
⚠️ File not found: 387_AUDIO.wav
⚠️ File not found: 388_AUDIO.wav
⚠️ File not found: 389_AUDIO.wav
⚠️ File not found: 390_AUDIO.wav
⚠️ File not found: 391_AUDIO.wav
⚠️ File not found: 392_AUDIO.wav
⚠️ File not found: 393_AUDIO.wav
⚠️ File not found: 394_AUDIO.wav
⚠️ File not found: 395_AUDIO.wav
⚠️ File not found: 396_AUDIO.wav
⚠️ File no

Copying transcripts to a single folder

(to be run only once)

In [7]:
!cp -r /content/transcripts /content/drive/MyDrive/CGS_Audio_FacialData/

cp: cannot stat '/content/transcripts': No such file or directory


# Predictor Label Generation


In [19]:
Depression = pd.read_csv('/content/drive/MyDrive/CGS_616_Facial_Transcripts_Audio_Data/metadata_mapped.csv')

Depression = Depression[['Participant_ID', 'PHQ_Binary']]
Depression_dict = dict(zip(Depression['Participant_ID'], Depression['PHQ_Binary']))

labels = []
transcript_path = '/content/drive/MyDrive/CGS_616_Facial_Transcripts_Audio_Data/transcripts'

for person_id in range(302, 440):
    transcript_file = os.path.join(transcript_path, f'{person_id}_transcript.csv')
    if os.path.exists(transcript_file):
        labels.append(Depression_dict.get(person_id, np.nan))

print(len(labels))
labels = np.array(labels)

105


# Extracting Feature Folders (Audio and Facial)

In [4]:
!unzip /content/drive/MyDrive/CGS_616_Facial_Transcripts_Audio_Data/audio_final.zip -d /content/
!unzip /content/drive/MyDrive/CGS_616_Facial_Transcripts_Audio_Data/all_csv_train.zip -d /content/

Archive:  /content/drive/MyDrive/CGS_Audio_FacialData/audio_final.zip
  inflating: /content/399_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/426_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/345_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/376_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/397_OpenSMILE2.3.0_egemaps.csv  
  inflating: /content/418_OpenSMILE2.3.0_egemaps.csv  
  inflating: /content/311_OpenSMILE2.3.0_egemaps.csv  
  inflating: /content/380_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/391_OpenSMILE2.3.0_egemaps.csv  
  inflating: /content/308_OpenSMILE2.3.0_egemaps.csv  
  inflating: /content/339_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/316_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/319_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/369_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/361_OpenSMILE2.3.0_egemaps.csv  
  inflating: /content/358_OpenSMILE2.3.0_egemaps.csv  
  inflating: /content/413_OpenSMILE2.3.0_mfcc.csv  
  inflating: /content/430

# Combining Audio Features

In [5]:
# Configuration for chunking utterances
CHUNK_SIZE = 15
CHUNK_OVERLAP = 4
MAX_SEQUENCE_LENGTH = 87

In [6]:
# Features from audio (eGeMAPS and MFCC)
# egemaps_features = ['F0semitoneFrom27.5Hz_sma3nz', 'HNRdBACF_sma3nz', 'jitterLocal_sma3nz',
#                     'shimmerLocaldB_sma3nz', 'Loudness_sma3', 'spectralFlux_sma3',
#                     'alphaRatio_sma3', 'logRelF0-H1-H2_sma3nz', 'F1frequency_sma3nz']

mfcc_features = ['pcm_fftMag_mfcc[2]', 'pcm_fftMag_mfcc[4]', 'pcm_fftMag_mfcc[7]',
                 'pcm_fftMag_mfcc[9]', 'pcm_fftMag_mfcc_de[2]', 'pcm_fftMag_mfcc_de[4]',
                 'pcm_fftMag_mfcc_de_de[2]', 'pcm_fftMag_mfcc_de_de[4]']

In [7]:
audio_features_combined = []

# Paths to transcript and audio feature files
transcripts_dir = '/content/drive/MyDrive/CGS_616_Facial_Transcripts_Audio_Data/transcripts'
audio_dir = '/content/'

# Loop through participants to extract audio features
for pid in range(302, 440):
    transcript_csv = os.path.join(transcripts_dir, f'{pid}_transcript.csv')
    if os.path.exists(transcript_csv):
        # egemaps_csv = os.path.join(audio_dir, f'{pid}_OpenSMILE2.3.0_egemaps.csv')
        mfcc_csv = os.path.join(audio_dir, f'{pid}_OpenSMILE2.3.0_mfcc.csv')

        transcript_df = pd.read_csv(transcript_csv)
        # egemaps_df = pd.read_csv(egemaps_csv, usecols=['frameTime'] + egemaps_features, sep=';')
        mfcc_df = pd.read_csv(mfcc_csv, usecols=['frameTime'] + mfcc_features, sep=';')

        feature_sequence = []
        chunked_data = []

        for _, utterance in transcript_df.iterrows():
            start, end = utterance['start'], utterance['end']
            # egemaps_seg = egemaps_df[(egemaps_df['frameTime'] >= start) & (egemaps_df['frameTime'] <= end)]
            mfcc_seg = mfcc_df[(mfcc_df['frameTime'] >= start) & (mfcc_df['frameTime'] <= end)]

            # pooled_egemaps = egemaps_seg[egemaps_features].max().values if not egemaps_seg.empty else np.zeros(len(egemaps_features))
            pooled_mfcc = mfcc_seg[mfcc_features].max().values if not mfcc_seg.empty else np.zeros(len(mfcc_features))
            feature_sequence.append(pooled_mfcc)

        for idx in range(0, len(feature_sequence) - CHUNK_SIZE + 1, CHUNK_SIZE - CHUNK_OVERLAP):
            chunk = np.vstack(feature_sequence[idx:idx + CHUNK_SIZE])
            chunked_data.append(chunk.max(axis=0))

        while len(chunked_data) < MAX_SEQUENCE_LENGTH:
            chunked_data.append(np.zeros(len(mfcc_features)))
        audio_features_combined.append(np.array(chunked_data[:MAX_SEQUENCE_LENGTH]))

In [8]:
# Normalize audio features
audio_features_combined = np.array(audio_features_combined)
audio_scaler = MinMaxScaler()
for f_idx in range(audio_features_combined.shape[2]):
    audio_features_combined[:, :, f_idx] = audio_scaler.fit_transform(audio_features_combined[:, :, f_idx])

# Combining Facial Feature

In [9]:
# Facial feature extraction
facial_features = ['pose_Rx', 'pose_Ry', 'pose_Rz', 'pose_Tx', 'pose_Ty', 'pose_Tz', 'gaze_0_x','gaze_0_y','gaze_0_z','gaze_1_x','gaze_1_y','gaze_1_z','gaze_angle_x', 'gaze_angle_y', 'AU04_r', 'AU05_r', 'AU07_r', 'AU14_r', 'AU15_r','AU20_r', 'AU25_r', 'AU26_r', 'AU45_r']

In [11]:
video_features_combined = []

for pid in range(302, 440):
    transcript_file = os.path.join(transcripts_dir, f'{pid}_transcript.csv')
    if os.path.exists(transcript_file):
        fau_csv = os.path.join(audio_dir, f'{pid}_OpenFace2.1.0_Pose_gaze_AUs.csv')

        transcript_df = pd.read_csv(transcript_file)
        fau_df = pd.read_csv(fau_csv, usecols=['timestamp'] + facial_features)

        feature_sequence = []
        chunked_data = []

        for _, row in transcript_df.iterrows():
            start, end = row['start'], row['end']
            segment = fau_df[(fau_df['timestamp'] >= start) & (fau_df['timestamp'] <= end)]
            pooled_features = segment[facial_features].max().values if not segment.empty else np.zeros(len(facial_features))
            feature_sequence.append(pooled_features)

        for idx in range(0, len(feature_sequence) - CHUNK_SIZE + 1, CHUNK_SIZE - CHUNK_OVERLAP):
            chunk = np.vstack(feature_sequence[idx:idx + CHUNK_SIZE])
            chunked_data.append(chunk.max(axis=0))

        while len(chunked_data) < MAX_SEQUENCE_LENGTH:
            chunked_data.append(np.zeros(len(facial_features)))
        video_features_combined.append(np.array(chunked_data[:MAX_SEQUENCE_LENGTH]))

In [12]:
video_features_combined = np.array(video_features_combined)
video_scaler = MinMaxScaler()
for f_idx in range(video_features_combined.shape[2]):
    video_features_combined[:, :, f_idx] = video_scaler.fit_transform(video_features_combined[:, :, f_idx])

# Combining and Generating Text Feature

In [13]:
# Text Embedding with SentenceTransformer
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
text_sequences = []

# Assuming `transcripts_data` is already populated
chunked_transcripts = {}
for pid in range(302, 440):
    path = os.path.join(transcripts_dir, f"{pid}_transcript.csv")
    if os.path.exists(path):
        df = pd.read_csv(path)
        utterances = df['text'].tolist()
        chunks = [" ".join(utterances[i:i + CHUNK_SIZE]) for i in range(0, len(utterances) - CHUNK_SIZE + 1, CHUNK_SIZE - CHUNK_OVERLAP)]
        chunked_transcripts[pid] = chunks

# Embed and pad text data
for pid, chunks in chunked_transcripts.items():
    embeddings = [embedding_model.encode(text) for text in chunks]
    text_sequences.append(embeddings)

text_sequences = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, dtype='float32', padding='post')

# Model : Facial features only

In [27]:
model_onlyfacial = Sequential([
    Input(shape=(MAX_SEQUENCE_LENGTH, len(facial_features))),
    Conv1D(32, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.3),
    Bidirectional(LSTM(32, return_sequences=False)),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model_onlyfacial.compile(optimizer=Adam(1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_onlyfacial.summary()

# Train model assuming `labels` are already defined
model_onlyfacial.fit(video_features_combined, labels, batch_size=32, epochs=15)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 87, 32)         │         2,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 87, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 87, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 64)             │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,065 (78.38 KB)

 Trainable params: 20,001 (78.13 KB)

 Non-trainable params: 64 (256.00 B)

Epoch 1/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 113ms/step - accuracy: 0.3943 - loss: 0.7483
Epoch 2/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.6850 - loss: 0.6440
Epoch 3/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.7256 - loss: 0.5956
Epoch 4/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.6715 - loss: 0.6318
Epoch 5/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.7229 - loss: 0.6029
Epoch 6/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.7423 - loss: 0.5922
Epoch 7/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.7274 - loss: 0.5570
Epoch 8/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.6993 - loss: 0.5796
Epoch 9/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.6934 - loss: 0.5976
Epoch 10/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.7433 - loss: 0.5968
Epoch 11/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.7076 - loss: 0.5950
Epoch 12/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7051 - loss

# MultiModal Analysis (Facial+Audio+Text)

In [16]:
# Combine all modalities
X_multimodal = np.concatenate([audio_features_combined, text_sequences, video_features_combined], axis=2)

In [17]:
# Binary classification model
multimodal_model = Sequential([
    Input(shape=(MAX_SEQUENCE_LENGTH, X_multimodal.shape[2])),
    Conv1D(64, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.3),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

multimodal_model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
multimodal_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 87, 64)         │        79,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 87, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 87, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 87, 128)        │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 150,721 (588.75 KB)

 Trainable params: 150,593 (588.25 KB)

 Non-trainable params: 128 (512.00 B)

In [18]:
# Train model assuming `labels` are already defined
multimodal_model.fit(X_multimodal, labels, batch_size=32, epochs=15)

Epoch 1/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 198ms/step - accuracy: 0.4360 - loss: 0.7209
Epoch 2/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step - accuracy: 0.6940 - loss: 0.6078
Epoch 3/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - accuracy: 0.7437 - loss: 0.5725
Epoch 4/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.6812 - loss: 0.6340
Epoch 5/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 203ms/step - accuracy: 0.7249 - loss: 0.5440
Epoch 6/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 188ms/step - accuracy: 0.7239 - loss: 0.5544
Epoch 7/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step - accuracy: 0.7051 - loss: 0.5570
Epoch 8/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.7020 - loss: 0.5056
Epoch 9/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - accuracy: 0.7409 - loss: 0.4424
Epoch 10/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - accuracy: 0.7783 - loss: 0.4400
Epoch 11/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.8418 - loss: 0.3765
Epoch 12/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.7940 - lo